In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
import cv2 as cv 
import glob
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler,Subset
from torchvision import transforms
from IPython.display import HTML
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torchvision  import datasets 
from sklearn.model_selection import KFold
import optuna
from optuna.trial import Trial
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn
from torchvision import models
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Subset, DataLoader

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
combined_data = "/kaggle/input/deepfake-and-real-images/Dataset/Test"

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),         # Convert to Tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize
])

In [6]:
combinedData = datasets.ImageFolder(combined_data, transform=transform)

In [7]:
combinedData

Dataset ImageFolder
    Number of datapoints: 10905
    Root location: /kaggle/input/deepfake-and-real-images/Dataset/Test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [8]:
kf = KFold(n_splits=3, shuffle=True, random_state=42) 

In [9]:
 b samples = len(combinedData)
samples

10905

In [10]:
folds = list(kf.split(np.arange(samples)))

In [11]:
folds

[(array([    1,     2,     4, ..., 10900, 10901, 10902]),
  array([    0,     3,     8, ..., 10899, 10903, 10904])),
 (array([    0,     1,     3, ..., 10902, 10903, 10904]),
  array([    2,    15,    18, ..., 10889, 10900, 10901])),
 (array([    0,     2,     3, ..., 10901, 10903, 10904]),
  array([    1,     4,     5, ..., 10896, 10897, 10902]))]

# Split our data into 3 Kfolds

In [12]:
for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
    print(f"Fold {fold_idx + 1}")
    
    train_size = int(0.75* len(train_val_idx)) 
    val_size = len(train_val_idx) - train_size #25
    
    train_idx = train_val_idx[:train_size]
    val_idx = train_val_idx[train_size:]
    
    # Create subsets
    train_set = Subset(combinedData, train_idx)
    val_set = Subset(combinedData, val_idx)
    test_set = Subset(combinedData, test_idx)
    
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
    
    print(f"Train size: {len(train_set)}")
    print(f"Validation size: {len(val_set)}")
    print(f"Test size: {len(test_set)}")

Fold 1
Train size: 5452
Validation size: 1818
Test size: 3635
Fold 2
Train size: 5452
Validation size: 1818
Test size: 3635
Fold 3
Train size: 5452
Validation size: 1818
Test size: 3635


## Applying Fine-tune

In [13]:
def get_model(name):
    if name == "vit_b_16":
        # Load the pretrained Vision Transformer model
        model = models.vit_b_16(pretrained=True)

        # Freeze all layers initially
        for param in model.parameters():
            param.requires_grad = False

        # Fine-tune the last transformer block
        for param in model.encoder.layers[-1].parameters():
            param.requires_grad = True

        # Modify the classifier for the desired number of output classes
        num_features = model.heads.head.in_features
        model.heads.head = nn.Linear(num_features, 2)

        # Ensure the classifier is trainable
        for param in model.heads.head.parameters():
            param.requires_grad = True

        # Print the number of trainable parameters
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Total trainable parameters: {trainable_params:,}")

    else:
        raise ValueError("Model name must be 'vit_b_16'")

    return model

## Training Function 

In [14]:
def TrainingModels(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)  #loss function used to compute the error between predictions and true labels
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1) # max score from the scores of two classes
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [15]:
def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

In [16]:
def LR_optimization(trial: Trial):
    # Use a smaller learning rate range for fine-tuning
    lr = trial.suggest_loguniform("lr", 1e-6, 1e-3)
    
    # Rest of the function remains the same
    fold_results = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(len(combinedData)))):  # Fixed parenthesis
        print(f"\nFold {fold_idx + 1}")
        
        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

        for model_name in ["vit_b_16"]:
            print(f"\nFine-tuning model: {model_name.upper()} on Fold {fold_idx + 1}")
            
            model = get_model(model_name).to(device)
            criterion = nn.CrossEntropyLoss()
            
            # Use a smaller weight decay for fine-tuning
            optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            
            # Add learning rate scheduler for fine-tuning
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='max', factor=0.5, patience=2, verbose=True
            )

            for epoch in range(5):
                current_lr = optimizer.param_groups[0]["lr"]
                print(f"\nEpoch {epoch + 1}/5 | Learning Rate: {current_lr:.6f}")
                
                train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
                
                # Evaluate after training each epoch
                val_loss, val_acc = evaluate(model, val_loader, criterion)

                # Update learning rate based on validation accuracy
                scheduler.step(val_acc)

                print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
                print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            
            fold_results.append(val_acc)

    mean_val_acc = sum(fold_results) / len(fold_results)
    return mean_val_acc


In [17]:
study = optuna.create_study(direction="maximize")
study.optimize(LR_optimization, n_trials=5)

best_trial = study.best_trial
print("\nBest Trial:")
print(f"  Value (Mean Validation Accuracy): {best_trial.value}")
print(f"  Params: {best_trial.params}")

[I 2024-12-28 15:54:41,237] A new study created in memory with name: no-name-1802b02a-e357-4435-8b52-85d6da53b3d0
<ipython-input-16-6cb39b1a2be4>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-6, 1e-3)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date wei


Fold 1

Fine-tuning model: VIT_B_16 on Fold 1


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:01<00:00, 194MB/s]  


Total trainable parameters: 7,089,410


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


...

Train Loss: 0.5886, Train Acc: 0.6878
Val Loss: 1.0079, Val Acc: 0.1689

Epoch 3/5 | Learning Rate: 0.000003


Train Loss: 0.5663, Train Acc: 0.7008
Val Loss: 0.9541, Val Acc: 0.2519

Epoch 4/5 | Learning Rate: 0.000003


Train Loss: 0.5479, Train Acc: 0.7117
Val Loss: 0.9499, Val Acc: 0.2827

Epoch 5/5 | Learning Rate: 0.000003


[I 2024-12-28 18:52:33,229] Trial 4 finished with value: 0.3384671800513384 and parameters: {'lr': 3.0948580585533394e-06}. Best is trial 1 with value: 0.759075907590759.


Train Loss: 0.5328, Train Acc: 0.7208
Val Loss: 0.9280, Val Acc: 0.3234

Best Trial:
  Value (Mean Validation Accuracy): 0.759075907590759
  Params: {'lr': 0.0005134706023131347}


# After the Optuna optimization process, we will use the best trial to set the learning rate

In [18]:
# Best learning rate from Optuna
best_lr = 0.0005134706023131347

In [19]:
final_results = {}

for model_name in ["vit_b_16"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")

    model = get_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=best_lr)

    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

    for epoch in range(10):
        print(f"\nEpoch {epoch + 1}/10")
        train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")

    test_loss, test_acc = evaluate(model, test_loader, criterion)
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

    final_results[model_name] = {
        "Test Loss": test_loss,
        "Test Accuracy": test_acc
    }

print("\nFinal Results:")
for model_name, metrics in final_results.items():
    print(f"{model_name.upper()}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")



Training model: VIT_B_16 with best learning rate 0.0005134706023131347
Total trainable parameters: 7,089,410

Epoch 1/10


Train Loss: 0.5065, Train Acc: 0.7428

Epoch 2/10


Train Loss: 0.3093, Train Acc: 0.8672

Epoch 3/10


Train Loss: 0.1925, Train Acc: 0.9184

Epoch 4/10


Train Loss: 0.1255, Train Acc: 0.9496

Epoch 5/10


...

Train Loss: 0.0405, Train Acc: 0.9850

Epoch 7/10


Train Loss: 0.0334, Train Acc: 0.9895

Epoch 8/10


Train Loss: 0.0233, Train Acc: 0.9930

Epoch 9/10


Train Loss: 0.0361, Train Acc: 0.9859

Epoch 10/10


Train Loss: 0.0125, Train Acc: 0.9952
Test Loss: 1.0857, Test Acc: 0.8223

Final Results:
VIT_B_16:
  Test Loss: 1.0857
  Test Accuracy: 0.8223


In [21]:
# Function to evaluate the model and get performance metrics
def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds), np.array(all_labels)

# Initialize cross-validation metrics storage
confusion_matrices = []
accuracies = []
precisions = []
recalls = []
f1_scores = []

best_lr = 0.0005134706023131347

for model_name in ["vit_b_16"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")
    
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"Fold {fold_idx + 1}")

        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
        
        
        model = get_model(model_name).to(device)
        optimizer = optim.Adam(model.parameters(), lr=best_lr)
        criterion = nn.CrossEntropyLoss()

        # Train the model
        for epoch in range(10):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            print(f"Epoch {epoch+1}/10, Loss: {running_loss / len(train_loader):.4f}")

    
        preds, labels = evaluate_model(model, test_loader)

        
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds, average='weighted', zero_division=0)
        recall = recall_score(labels, preds, average='weighted', zero_division=0)
        f1 = f1_score(labels, preds, average='weighted', zero_division=0)
        cm = confusion_matrix(labels, preds)

        # Store metrics for the fold
        confusion_matrices.append(cm)
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        print(f"Fold {fold_idx + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
        print(f"Confusion Matrix:\n{cm}\n")

    # Average metrics across folds
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1_score = np.mean(f1_scores)
    avg_confusion_matrix = np.mean(confusion_matrices, axis=0)

    # Print final average metrics for the model
    print(f"\nFinal Results for {model_name.upper()}:")
    print(f"Average Accuracy: {avg_accuracy:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1-Score: {avg_f1_score:.4f}")
    print(f"Average Confusion Matrix:\n{avg_confusion_matrix}")


Training model: VIT_B_16 with best learning rate 0.0005134706023131347
Fold 1
Total trainable parameters: 7,089,410
Epoch 1/10, Loss: 0.5391
Epoch 2/10, Loss: 0.3486
Epoch 3/10, Loss: 0.2191
Epoch 4/10, Loss: 0.1267
Epoch 5/10, Loss: 0.0732
Epoch 6/10, Loss: 0.0671
Epoch 7/10, Loss: 0.0462
Epoch 8/10, Loss: 0.0317
Epoch 9/10, Loss: 0.0115
Epoch 10/10, Loss: 0.0363
Fold 1 - Accuracy: 0.8080, Precision: 0.8308, Recall: 0.8080, F1-Score: 0.8037
Confusion Matrix:
[[1759  105]
 [ 593 1178]]

Fold 2


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Total trainable parameters: 7,089,410
Epoch 1/10, Loss: 0.5174
Epoch 2/10, Loss: 0.3144
Epoch 3/10, Loss: 0.1746
Epoch 4/10, Loss: 0.1094
Epoch 5/10, Loss: 0.0523
Epoch 6/10, Loss: 0.0500
Epoch 7/10, Loss: 0.0351
Epoch 8/10, Loss: 0.0321
Epoch 9/10, Loss: 0.0233
Epoch 10/10, Loss: 0.0203
Fold 2 - Accuracy: 0.8072, Precision: 0.8235, Recall: 0.8072, F1-Score: 0.8044
Confusion Matrix:
[[1690  144]
 [ 557 1244]]

Fold 3


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Total trainable parameters: 7,089,410
Epoch 1/10, Loss: 0.5146
Epoch 2/10, Loss: 0.3499
Epoch 3/10, Loss: 0.2068
Epoch 4/10, Loss: 0.1364
Epoch 5/10, Loss: 0.0741
Epoch 6/10, Loss: 0.0643
Epoch 7/10, Loss: 0.0348
Epoch 8/10, Loss: 0.0235
Epoch 9/10, Loss: 0.0279
Epoch 10/10, Loss: 0.0164
Fold 3 - Accuracy: 0.8085, Precision: 0.8199, Recall: 0.8085, F1-Score: 0.8071
Confusion Matrix:
[[1614  180]
 [ 516 1325]]


Final Results for VIT_B_16:
Average Accuracy: 0.8079
Average Precision: 0.8247
Average Recall: 0.8079
Average F1-Score: 0.8051
Average Confusion Matrix:
[[1687.66666667  143.        ]
 [ 555.33333333 1249.        ]]
